# Exploratory Analysis

## Eploring SpotiPy
- Explore available data
- Select features of interest
- Generate initial dataframe and database

###### Note:  Potentially building a recomender system that can take a set of the most frequently played songs from one user, and match them with a second users profile; potentially between courting couples and friends

In [41]:
# imports

import os
import sys
import json
import spotipy
import webbrowser
from os import getenv
import spotipy.util as util
from dotenv import load_dotenv
from json.decoder import JSONDecodeError
from spotipy.oauth2 import SpotifyClientCredentials

In [42]:
# We are using the client Module from the Python library for the Spotify API
# (https://spotipy.readthedocs.io/en/2.13.0/#module-spotipy.client)
# Client Credentials Flow


load_dotenv()  # this imports all .env variables

# Setting up env variables to connect to API
uri = getenv('uri') # must match in the Spotify app dashboard
SPOTIFY_CLIENT_ID = getenv('SPOTIFY_CLIENT_ID')
SPOTIFY_CLIENT_SECRET = getenv('SPOTIFY_CLIENT_SECRET')
username = getenv('USER_ID')  #  user who's data we are collecting
scope = 'user-top-read'  #  determines the kind of access you have to a user profile

# Access token to obtain user info
token = util.prompt_for_user_token(username=username,
                                   scope=scope,
                                   client_id=SPOTIFY_CLIENT_ID,
                                   client_secret=SPOTIFY_CLIENT_SECRET,
                                   redirect_uri=uri)

# activating spotify session
spotify_session = spotipy.Spotify(auth=token)

## Goals 
- Connect to user library using [scopes](https://developer.spotify.com/documentation/general/guides/scopes/)
- Scopes to connect to are [user-library-read](https://developer.spotify.com/documentation/general/guides/scopes/#user-library-read), [playlist-modify-public](https://developer.spotify.com/documentation/general/guides/scopes/#playlist-modify-public), and [user-top-read](https://developer.spotify.com/documentation/general/guides/scopes/#user-top-read)

##### *The goal here is to cnnect to the user's library, analyze it, and create a library.  One thing to keep in mind is that we will need user_1 and user_2 libraries*

In [80]:
# Playing with the api: accessing user top read, modify playlist, read library

# User top artists
top_artists = spotify_session.current_user_top_artists(limit=1)

# User top tracks
top_tracks = spotify_session.current_user_top_tracks(limit=1)

# Track_ids
top_tracks['items'][0]['id'] == "0akyEssGRVHstqCSWXusJL"
top_track_id = top_tracks['items'][0]['id']

# Track features
audio_anal = spotify_session.audio_analysis(top_track_id)

In [92]:
# See the following link for discriptive information regarding data generated
# by the .audio_analysis() method.
# (https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-analysis/#time-interval-object)

audio_anal['bars']   # Segment of time defined as given number of beats
audio_anal['beats']  # time intervals of beats throughout track
audio_anal['']

[{'start': 0.8118, 'duration': 0.55077, 'confidence': 0.558},
 {'start': 1.36257, 'duration': 0.60501, 'confidence': 0.506},
 {'start': 1.96758, 'duration': 0.59686, 'confidence': 0.064},
 {'start': 2.56444, 'duration': 0.58629, 'confidence': 0.778},
 {'start': 3.15073, 'duration': 0.59285, 'confidence': 0.84},
 {'start': 3.74358, 'duration': 0.58402, 'confidence': 0.722},
 {'start': 4.32761, 'duration': 0.57836, 'confidence': 0.827},
 {'start': 4.90597, 'duration': 0.57421, 'confidence': 0.847},
 {'start': 5.48018, 'duration': 0.57261, 'confidence': 0.819},
 {'start': 6.0528, 'duration': 0.57, 'confidence': 0.665},
 {'start': 6.6228, 'duration': 0.57199, 'confidence': 0.844},
 {'start': 7.19479, 'duration': 0.57295, 'confidence': 0.856},
 {'start': 7.76774, 'duration': 0.57243, 'confidence': 0.862},
 {'start': 8.34017, 'duration': 0.56834, 'confidence': 0.763},
 {'start': 8.90851, 'duration': 0.56936, 'confidence': 0.846},
 {'start': 9.47787, 'duration': 0.56885, 'confidence': 0.819},

# Baseline Explorations
- Select type of problem type (class/reg)
- Determine model baselines
- Model evaluations/comparisons